In [127]:
import os

import numpy as np
from PIL import Image
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_array, check_is_fitted, check_X_y
from sklearn.metrics import accuracy_score

In [100]:
def load_png_images(folder_path, target_size=(256, 256)):
    """
    Loads all PNG images from a folder into a NumPy array.

    Args:
      folder_path: Path to the folder containing PNG images.

    Returns:
      A NumPy array with shape (num_images, height, width, channels) containing
      the loaded images.
    """
    images = []

    for filename in os.listdir(folder_path):
        if not filename.endswith(".jpg"):
            continue
        img = Image.open(os.path.join(folder_path, filename))
        img = img.resize(target_size)
        img_array = np.array(img)
        images.append(img_array)
    # Stack images into a NumPy array
    images_np = np.stack(images)
    return images_np

In [101]:
PATH = "./plates"

In [125]:
cleaned = load_png_images(f"{PATH}/train/cleaned")
dirty = load_png_images(f"{PATH}/train/dirty")

### Criar as labels para as images limpas(0) e sujas(1) 

In [103]:
labels = np.concatenate(
    [
        np.zeros(shape=(cleaned.shape[0],), dtype=np.int8),
        np.ones(shape=(dirty.shape[0],), dtype=np.int8),
    ]
)

### Funções para transformar uma imagem (256,256,3) em (196608, )

In [104]:
def train_test_split(data, test_size=0.2):
    split_idx = int(len(data) * (1 - test_size))

    train_data = data[:split_idx]
    test_data = data[split_idx:]

    X_train = np.array([item[0] for item in train_data])
    Y_train = np.array([item[1] for item in train_data]).astype(int)
    X_test = np.array([item[0] for item in test_data])
    Y_test = np.array([item[1] for item in test_data]).astype(int)

    return X_train, Y_train, X_test, Y_test


def flatten_images(data):
    flattened_data = []
    for features, label in data:
        flattened_features = features.flatten()
        flattened_data.append((flattened_features, label))
    return flattened_data

### Normalizar e separar dados de treino e teste

In [105]:
data = np.concatenate([cleaned, dirty]).astype("float32") / 255.0
data = list(zip(data, labels))
np.random.shuffle(data)
data = flatten_images(data)
X_train, y_train, X_test, y_test = train_test_split(data)

X_train.shape, y_train.shape

((32, 196608), (32,))

In [118]:
class NeuralNetwork(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_units=128, learning_rate=0.01, epochs=1000):
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.scaler = StandardScaler()

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _initialize_parameters(self, input_size):
        np.random.seed(0)
        self.W1 = np.random.randn(input_size, self.hidden_units)
        self.b1 = np.zeros((1, self.hidden_units))
        self.W2 = np.random.randn(self.hidden_units, 1)
        self.b2 = np.zeros((1, 1))

    def _forward_propagation(self, X):
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = np.tanh(Z1)
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = self._sigmoid(Z2)
        return A1, A2

    def _backward_propagation(self, X, y, A1, A2):
        m = X.shape[0]
        dZ2 = A2 - y.reshape(-1, 1)
        dW2 = (1 / m) * np.dot(A1.T, dZ2)
        db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)
        dZ1 = np.dot(dZ2, self.W2.T) * (1 - np.power(A1, 2))
        dW1 = (1 / m) * np.dot(X.T, dZ1)
        db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)
        return dW1, db1, dW2, db2

    def _update_parameters(self, dW1, db1, dW2, db2):
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.input_size_ = X.shape[1]
        self._initialize_parameters(self.input_size_)

        for epoch in range(self.epochs):
            A1, A2 = self._forward_propagation(X)
            dW1, db1, dW2, db2 = self._backward_propagation(X, y, A1, A2)
            self._update_parameters(dW1, db1, dW2, db2)

        return self

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        A1, A2 = self._forward_propagation(X)
        return A2

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return (probabilities > 0.5).astype(int).flatten()

In [119]:
# Assuming X_train and y_train are your training data
input_size = X_train.shape[1]
hidden_layers = [128, 64, 32]  # Example of adding a hidden layer
model = NeuralNetwork()
model.fit(X_train, y_train)

NeuralNetwork()

In [126]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5
